In [68]:
import numpy as np
import pandas as pd
import ast
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np
import pandas as pd
import ast
from gensim.models import Word2Vec

In [69]:
credits_data = pd.read_csv("C:/Users/srina/Documents/Machine learning/Project/credits.csv")
movies_data = pd.read_csv("C:/Users/srina/Documents/Machine learning/Project/updated_movies.csv")
users_data = pd.read_csv("C:/Users/srina/Documents/Machine learning/Project/user1.csv")

In [70]:
combined_movie_data = movies_data.merge(credits_data, on='title')
combined_movie_data = combined_movie_data[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew', 'emotion']]

In [71]:
combined_movie_data.head()

,movie_id,title,overview,genres,keywords,cast,crew,emotion
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Joy
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Disgust
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Sadness
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Inspiration
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",Joy


In [72]:
users_data.head()

,user_id,movie_id,rating
0,user_id_1,19995,3
1,user_id_1,285,2
2,user_id_1,206647,3
3,user_id_1,49026,5
4,user_id_1,49529,3


In [73]:
def parse_and_extract_names(stringified_list):
    extracted_names = []
    for item in ast.literal_eval(stringified_list):
        extracted_names.append(item['name'])
    return extracted_names

In [74]:
combined_movie_data.dropna(inplace=True)
combined_movie_data['genres'] = combined_movie_data['genres'].apply(parse_and_extract_names)
combined_movie_data['keywords'] = combined_movie_data['keywords'].apply(parse_and_extract_names)

In [75]:
def extract_top_three_cast(stringified_list):
    top_cast = []
    for i, item in enumerate(ast.literal_eval(stringified_list)):
        if i < 3:
            top_cast.append(item['name'])
    return top_cast

In [76]:
combined_movie_data['cast'] = combined_movie_data['cast'].apply(extract_top_three_cast)


In [77]:
def extract_director_name(directors_list):
    for item in ast.literal_eval(directors_list):
        if item['job'] == 'Director':
            return [item['name']]
    return []

In [78]:
combined_movie_data['crew'] = combined_movie_data['crew'].apply(extract_director_name)

In [79]:
combined_movie_data['overview'] = combined_movie_data['overview'].apply(lambda x: x.split())
combined_movie_data['emotion'] = combined_movie_data['emotion'].apply(lambda x: x.lower().replace(" ", ""))

In [80]:
combined_movie_data['tags'] = combined_movie_data['overview'] + combined_movie_data['genres'] + combined_movie_data['keywords'] + combined_movie_data['cast'] + combined_movie_data['crew'] + combined_movie_data['emotion'].apply(lambda x: [x])
combined_movie_data['tags'] = combined_movie_data['tags'].apply(lambda x: ' '.join(x))
combined_movie_data = combined_movie_data.drop_duplicates(subset='title', keep='first')


In [ ]:
combined_movie_data.head()

In [81]:
combined_movie_data1 = combined_movie_data
combined_movie_data2 = combined_movie_data 
users_data1 = users_data
users_data2 = users_data


**Content Based Filtereing**

In [82]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(combined_movie_data1['tags']).toarray()

# Dimensionality Reduction
svd = TruncatedSVD(n_components=100, random_state=42)
vectors = svd.fit_transform(vectors)

In [83]:
# Filter the input user's movies based on the specified emotion

def recommend_movies_based_on_emotion_rating_and_other_users(input_user_id, emotion, user_rating_threshold=4, n_movies=5):
    
    input_user_movies = users_data1[users_data1['user_id'] == input_user_id]
    input_user_merged_movies = combined_movie_data1.merge(input_user_movies, on='movie_id')

    input_user_emotion_movies = input_user_merged_movies[input_user_merged_movies['emotion'].str.lower() == emotion.lower()]

    input_user_emotion_movies = input_user_emotion_movies[input_user_emotion_movies['rating'] >= user_rating_threshold]

    input_user_emotion_movies['tags'] = input_user_emotion_movies['tags'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

    input_user_emotion_vectors = svd.transform(cv.transform(input_user_emotion_movies['tags']))

    recommendations = []
    for idx, emotion_vector in enumerate(input_user_emotion_vectors):
        sim_scores = list(enumerate(cosine_similarity([emotion_vector], vectors)[0]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        for movie_idx, sim_score in sim_scores[1:]:
            movie_id = combined_movie_data1.iloc[movie_idx]['movie_id']
            other_users_ratings = users_data1[(users_data1['movie_id'] == movie_id) & (users_data1['user_id'] != input_user_id) & (users_data1['rating'] >= user_rating_threshold)]
            
            for _, user_rating in other_users_ratings.iterrows():
                movie_emotion = combined_movie_data1.iloc[movie_idx]['emotion'].lower()
                # Strictly add only those movies that exactly match the specified emotion
                if movie_emotion == emotion.lower():
                    recommendations.append((combined_movie_data1.iloc[movie_idx]['title'], sim_score, user_rating['user_id'], user_rating['rating']))

    unique_recommendations = sorted(list(set(recommendations)), key=lambda x: x[1], reverse=True)[:n_movies]

    return unique_recommendations


In [84]:
user_id = "user_id_2"  
emotion = "Anger"  
recommended_movies = recommend_movies_based_on_emotion_rating_and_other_users(user_id, emotion)


for movie, sim_score, other_user_id, rating in recommended_movies:
    print(f"Movie: {movie} ---------- Similarity percentage: {sim_score*100:.2f}%  ----------- Rating: {rating}")


Movie: The Devil's Own ---------- Similarity percentage: 68.62%  ----------- Rating: 5
Movie: Stonewall ---------- Similarity percentage: 65.31%  ----------- Rating: 4
Movie: Blindness ---------- Similarity percentage: 63.02%  ----------- Rating: 5
Movie: Charly ---------- Similarity percentage: 60.93%  ----------- Rating: 4
Movie: The Cell ---------- Similarity percentage: 60.02%  ----------- Rating: 5


**Hybrid model(collaborative filtering with a predictive model)**

In [85]:
#Converting categorial values to numerical values
combined_movie_data2['tags'] = combined_movie_data2['tags'].apply(lambda x: x.split())
word2vec_model = Word2Vec(combined_movie_data2['tags'], vector_size=100, window=5, min_count=2, workers=4)

In [86]:
def create_movie_vector(words, model):
    valid_words = [word for word in words if word in model.wv]
    if not valid_words:
        return np.zeros(model.vector_size)
    word_vectors = np.array([model.wv[word] for word in valid_words])
    vector = np.mean(word_vectors, axis=0)
    return vector

movie_id_to_vector = {movie_id: create_movie_vector(words, word2vec_model) for movie_id, words in zip(combined_movie_data2['movie_id'], combined_movie_data2['tags'])}

In [87]:
#setting up training data
average_ratings = users_data2.groupby('user_id')['rating'].mean()

X = []
y = []

for _, row in users_data2.iterrows():
    user_id = row['user_id']
    movie_id = row['movie_id']
    rating = row['rating']
    
    user_feature = np.array([average_ratings.get(user_id, 0), 0])  
    if movie_id in movie_id_to_vector:
        movie_feature = movie_id_to_vector[movie_id]
        feature_vector = np.concatenate((user_feature, movie_feature))
        X.append(feature_vector)
        y.append(1 if rating >= 4 else 0)  

X = np.array(X)
y = np.array(y)



In [88]:
# Split the data for a RandomForest model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [89]:
# Function to recommend movies based on other users' high ratings and specific emotion
def recommend_movies_based_on_other_users_high_ratings(user_id, emotion, n_movies=5):
    
    high_rated_movies = users_data2[users_data2['rating'] >= 4]
    emotion_movies = combined_movie_data2[combined_movie_data2['emotion'].str.contains(emotion, case=False)]
    high_rated_emotion_movies = high_rated_movies[high_rated_movies['movie_id'].isin(emotion_movies['movie_id'])]

    high_rated_emotion_movies = high_rated_emotion_movies[high_rated_emotion_movies['user_id'] != user_id]

    predictions = []
    for _, movie_row in high_rated_emotion_movies.iterrows():
        movie_id = movie_row['movie_id']
        if movie_id in movie_id_to_vector:
            movie_feature = movie_id_to_vector[movie_id]
            user_feature = np.array([average_ratings.get(user_id, 0), 0])  
            feature_vector = np.concatenate((user_feature, movie_feature)).reshape(1, -1)
            predicted_like = model.predict_proba(feature_vector)[0][1]
            movie_title = combined_movie_data2[combined_movie_data2['movie_id'] == movie_id]['title'].iloc[0]
            predictions.append((movie_title, predicted_like))

    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions[:n_movies]


In [100]:
recommended_movies = recommend_movies_based_on_other_users_high_ratings('user_id_8', 'Joy')
for movie, score in recommended_movies:
    print(f"Movie: {movie} ----------------------------- Predicted Preference: {score}")


Movie: Blonde Ambition ----------------------------- Predicted Preference: 0.88
Movie: Just Go with It ----------------------------- Predicted Preference: 0.87
Movie: Raising Helen ----------------------------- Predicted Preference: 0.87
Movie: I Married a Strange Person! ----------------------------- Predicted Preference: 0.85
Movie: The Sentinel ----------------------------- Predicted Preference: 0.84
